In [6]:

import requests
from requests import Response
import pandas as pd

from pydantic import BaseModel, RootModel, Field

class BikeOfSite(BaseModel):
    行政區:str = Field(alias='sarea')
    站點名稱:str = Field(alias='sna')
    時間:str = Field(alias='mday')
    總車輛數:int = Field(alias='tot')
    可借:int = Field(alias='sbi')
    可還:int = Field(alias='bemp')

############################

class BikeOfSites(RootModel):
    root:list[BikeOfSite]

    def __iter__(self):
        return iter(self.root)
    
    def __getitem__(self, idx):
        return self.root[idx]

############################

def get_response() -> Response | None:
    url_path:str = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'
    print(f'開始下載:')

    try:
        response:Response = requests.get(url_path, timeout=15)
        response.raise_for_status()
    except Exception as e:
        print('連線失敗。')
        print(e)
        return None
    
    if response.status_code != 200:
        print('下載失敗。')
        print(response.reason)
        return None
    
    print('下載成功')
    return response
  

############################

def jsonToList(raw:str) -> list[dict] | None:
    try:
        datas:BikeOfSites = BikeOfSites.model_validate_json(raw)
        items:list[dict] = datas.model_dump()
        return items
    except Exception as e:
        print('預想外的格式。')
        print(e)
        return None

############################

def main():
    response:Response | None = get_response()

    if not response:
        return
    
    rawString:str = response.text
    download_data:list[dict] | None = jsonToList(rawString)

    if not download_data:
        return

    df = pd.DataFrame(download_data)

    display(df)
    
main()



開始下載:
下載成功


,行政區,站點名稱,時間,總車輛數,可借,可還
0,大安區,YouBike2.0_捷運科技大樓站,2024-04-15 15:45:19,28,7,21
1,大安區,YouBike2.0_復興南路二段273號前,2024-04-15 15:43:18,21,2,19
2,大安區,YouBike2.0_國北教大實小東側門,2024-04-15 15:45:19,16,1,15
3,大安區,YouBike2.0_和平公園東側,2024-04-15 15:45:19,11,1,10
4,大安區,YouBike2.0_辛亥復興路口西北側,2024-04-15 15:40:30,16,3,13
...,...,...,...,...,...,...
1401,臺大公館校區,YouBike2.0_臺大總圖書館西南側,2024-04-15 15:36:13,30,20,1
1402,臺大公館校區,YouBike2.0_臺大黑森林西側,2024-04-15 15:46:19,20,9,11
1403,臺大公館校區,YouBike2.0_臺大獸醫館南側,2024-04-15 15:44:19,24,7,12
1404,臺大公館校區,YouBike2.0_臺大新體育館東南側,2024-04-15 15:46:19,40,30,2
